<a href="https://colab.research.google.com/github/lebe1/text-oriented-data-science-project/blob/main/Data_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the Dataset

## Connect to Google Drive

In [16]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [17]:
folder_path = '/content/drive/MyDrive/'

## Imports

In [107]:
import pandas as pd
import numpy as np
import nltk
import time
from nltk.corpus import stopwords
import string
from gensim.models import Word2Vec

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Reading the CSV File

In [33]:
file_name = 'combined_reviews.csv'

file_path = folder_path + file_name
df = pd.read_csv(file_path)

In [34]:
df.head()

,rating,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,reviewToken
0,5.0,2017-01-16,ASWLL1VJA7WOG,Great product... just what I wanted. Works gr...,Five Stars,1484524800,All_Beauty,"['great', 'product', 'want', 'works', 'great',..."
1,5.0,2008-12-08,A265K3A7V83112,"After seeing the popularity of this shoe, I de...",What can i say? chucks rock,1228694400,Clothing_Shoes_and_Jewelry,"['see', 'popularity', 'shoe', 'decide', 'test'..."
2,5.0,2013-02-08,A1D18EJF6LHYDV,I was nervousness about the scent because IVe ...,Smells great,1360281600,All_Beauty,"['nervousness', 'scent', 'ive', 'never', 'try'..."
3,5.0,2018-02-15,A25EOTX5I354I2,"I LOVE the smell. A bit expensive, so I cant b...",Five Stars,1518652800,Luxury_Beauty,"['love', 'smell', 'bit', 'expensive', 'buy', '..."
4,5.0,2013-11-11,A1DFZPQPCHBYTY,Found this stuff in Japan and wondered if I co...,Super lathery nice soap!,1384128000,All_Beauty,"['found', 'stuff', 'japan', 'wonder', 'could',..."


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rating          12000 non-null  float64
 1   reviewTime      12000 non-null  object 
 2   reviewerID      12000 non-null  object 
 3   reviewText      11975 non-null  object 
 4   summary         11991 non-null  object 
 5   unixReviewTime  12000 non-null  int64  
 6   category        12000 non-null  object 
 7   reviewToken     12000 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 750.1+ KB


# Build the model

In [53]:
# Preprocessing
def preprocess_text(text):
    # Convert only string instances to lowercase
    text = text.lower() if isinstance(text, str)  else ''
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = text.split()
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)


df['preprocessedText'] = df['reviewText'].apply(preprocess_text)

# Tokenize text again for word2vec
df['tokenized_text'] = df['preprocessedText'].str.split()

df['preprocessedText'].head()


,preprocessedText
0,great product wanted works great stylish
1,seeing popularity shoe decided test impressed ...
2,nervousness scent ive never tried love paul mi...
3,love smell bit expensive cant buy often would ...
4,found stuff japan wondered could find 3drops g...


In [74]:
# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=100)
X_tfidf = tfidf.fit_transform(df['preprocessedText']).toarray()

# Word2Vec Embeddings
w2v_model = Word2Vec(sentences=df['tokenized_text'], vector_size=100, window=5, min_count=1, workers=4)
def get_sentence_embedding(word_list):
    word_vecs = [w2v_model.wv[word] for word in word_list if word in w2v_model.wv]
    if word_vecs:
        return np.mean(word_vecs, axis=0)
    else:
        return np.zeros(w2v_model.vector_size)

X_w2v = np.array([get_sentence_embedding(word_list) for word_list in df['tokenized_text']])


In [108]:
# Combine TF-IDF and Word2Vec features
X_combined = np.hstack((X_tfidf, X_w2v))
y = df['rating']

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

start_time = time.time()
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
end_time = time.time()
execution_time = end_time - start_time

f1Score = f1_score(y_test, y_pred_rf, average='macro')
precision = precision_score(y_test, y_pred_rf, average='macro')
recall = recall_score(y_test, y_pred_rf, average='macro')

print("Execution Time:", execution_time, "seconds")
print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1Score,"\n")
print(classification_report(y_test, y_pred_rf))

Execution Time: 12.419667720794678 seconds
Precision Score: 0.8093630344410183
Recall Score: 0.49085787087576105
F1 Score: 0.5823812445362928 

              precision    recall  f1-score   support

         1.0       0.79      0.32      0.46        68
         2.0       0.84      0.37      0.51        73
         3.0       0.86      0.40      0.55       182
         4.0       0.74      0.38      0.50       351
         5.0       0.82      0.98      0.89      1726

    accuracy                           0.81      2400
   macro avg       0.81      0.49      0.58      2400
weighted avg       0.81      0.81      0.78      2400



In [109]:
start_time = time.time()
linear_svc_model = LinearSVC(random_state=42)
linear_svc_model.fit(X_train, y_train)

y_pred_svc = linear_svc_model.predict(X_test)
end_time = time.time()
execution_time = end_time - start_time

f1Score = f1_score(y_test, y_pred_svc, average='macro')
precision = precision_score(y_test, y_pred_svc, average='macro')
recall = recall_score(y_test, y_pred_svc, average='macro')

print("Execution Time:", execution_time, "seconds")
print("Precision Score:", precision)
print("Recall Score:", recall)
print("F1 Score:", f1Score, "\n")
print(classification_report(y_test, y_pred_svc))

Execution Time: 6.5763795375823975 seconds
Precision Score: 0.5477349105920535
Recall Score: 0.2999865385803818
F1 Score: 0.3314572005651647 

              precision    recall  f1-score   support

         1.0       0.41      0.16      0.23        68
         2.0       0.50      0.04      0.08        73
         3.0       0.54      0.15      0.24       182
         4.0       0.53      0.17      0.25       351
         5.0       0.77      0.98      0.86      1726

    accuracy                           0.74      2400
   macro avg       0.55      0.30      0.33      2400
weighted avg       0.70      0.74      0.68      2400



## TODO
- implement wandb
- qualitative analysis on 2-3 misclassifications
- optimizing model by balancing data set